In [26]:
pip install nltk


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [27]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/c8/w8kx66gd59z1pfs7k6npw6d80000gn/T/ipykernel_5104/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [29]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [30]:
# Your code
from sklearn.model_selection import train_test_split

# Assuming the target column is 'label' and the text column is 'text'
X = data['text']
y = data['label']

# Split the data into training and validation sets (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ghadam/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [41]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [43]:
# Your code
import re

def clean_html(text):
    # Remove inline JavaScript/CSS
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL)
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL)
    
    # Remove HTML comments (must be done before removing regular tags)
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    
    # Remove remaining HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    return text

# Example usage
html_text = """
<html>
<head><title>Test Page</title></head>
<body>
<script type="text/javascript">alert("Hello!");</script>
<style>body { background-color: white; }</style>
<!-- This is a comment -->
<p>This is a <b>test</b> paragraph.</p>
</body>
</html>
"""

cleaned_text = clean_html(html_text)
print(cleaned_text)



Test Page




This is a test paragraph.





- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [45]:
# Your code
import re

def clean_text(text):
    # Remove special characters (keep only alphabets and spaces)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove all single characters (e.g., 'a', 'b', 'c')
    text = re.sub(r'\b\w\b', '', text)
    
    # Remove single characters from the start
    text = re.sub(r'^\w\s+', '', text)
    
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove prefixed 'b' (common in byte strings)
    text = re.sub(r'^b\s+', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text.strip()  # Remove leading/trailing whitespace

# Example usage
dirty_text = "b This is a 123 test! @#$% with some single a b c characters and   multiple   spaces."

cleaned_text = clean_text(dirty_text)
print(cleaned_text)

this is test with some single characters and multiple spaces


## Now let's work on removing stopwords
Remove the stopwords.

In [46]:
# Your code
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

X_train = X_train.apply(remove_stopwords)
X_val = X_val.apply(remove_stopwords)

In [48]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/ghadam/nltk_data...


True

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [49]:
# Your code
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

X_train = X_train.apply(lemmatize_text)
X_val = X_val.apply(lemmatize_text)

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Assuming 'data' is your DataFrame with 'text' and 'label' columns
# 'label' column should have 'ham' or 'spam' as values

# Separate ham and spam messages
ham_messages = data[data['label'] == 'ham']['text']
spam_messages = data[data['label'] == 'spam']['text']

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform ham messages
ham_word_counts = vectorizer.fit_transform(ham_messages)

# Fit and transform spam messages
spam_word_counts = vectorizer.fit_transform(spam_messages)

# Sum the counts of each word in ham and spam
ham_word_freq = pd.DataFrame(ham_word_counts.toarray(), columns=vectorizer.get_feature_names_out()).sum().sort_values(ascending=False)
spam_word_freq = pd.DataFrame(spam_word_counts.toarray(), columns=vectorizer.get_feature_names_out()).sum().sort_values(ascending=False)

# Get the top 10 words for ham and spam
top_10_ham_words = ham_word_freq.head(10)
top_10_spam_words = spam_word_freq.head(10)

print("Top 10 words in HAM messages:")
print(top_10_ham_words)

print("\nTop 10 words in SPAM messages:")
print(top_10_spam_words)

## Extra features

In [51]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

NameError: name 'data_train' is not defined

## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer

# Sample documents
documents = [
    "I love NLP.",
    "I love coding and NLP."
]

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the documents
X = vectorizer.fit_transform(documents)

# Convert to array
X_array = X.toarray()

# Get feature names (words in the vocabulary)
feature_names = vectorizer.get_feature_names_out()

# Create a DataFrame for visualization
import pandas as pd
df = pd.DataFrame(X_array, columns=feature_names)
print(df)

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [54]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform the validation data
X_val_tfidf = tfidf_vectorizer.transform(X_val)

# Print the shape of the vectorized dataset
print(X_train_tfidf.shape)
print(X_val_tfidf.shape)

(800, 23064)
(200, 23064)


## And the Train a Classifier?

In [55]:
# Your code
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Initialize the Multinomial Naive Bayes classifier
classifier = MultinomialNB()

# Train the classifier on the TF-IDF transformed training data
classifier.fit(X_train_tfidf, y_train)

# Predict on the validation set
y_pred = classifier.predict(X_val_tfidf)

# Evaluate the classifier
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Classification Report:\n", classification_report(y_val, y_pred))

Accuracy: 0.93
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.89      0.94       125
           1       0.84      1.00      0.91        75

    accuracy                           0.93       200
   macro avg       0.92      0.94      0.93       200
weighted avg       0.94      0.93      0.93       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [56]:
# Your code
# Example of adding extra features
data_train = pd.DataFrame({'preprocessed_text': X_train, 'label': y_train})
data_val = pd.DataFrame({'preprocessed_text': X_val, 'label': y_val})

money_simbol_list = "|".join(["euro", "dollar", "pound", "€", "$"])
suspicious_words = "|".join(["free", "cheap", "sex", "money", "account", "bank", "fund", "transfer", "transaction", "win", "deposit", "password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list).astype(int)
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words).astype(int)
data_train['text_len'] = data_train['preprocessed_text'].apply(len)

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list).astype(int)
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words).astype(int)
data_val['text_len'] = data_val['preprocessed_text'].apply(len)

# Combine TF-IDF features with extra features
from scipy.sparse import hstack

X_train_combined = hstack([X_train_tfidf, data_train[['money_mark', 'suspicious_words', 'text_len']]])
X_val_combined = hstack([X_val_tfidf, data_val[['money_mark', 'suspicious_words', 'text_len']]])

# Train the classifier on the combined features
classifier.fit(X_train_combined, y_train)

# Predict on the validation set
y_pred = classifier.predict(X_val_combined)

# Evaluate the classifier
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Classification Report:\n", classification_report(y_val, y_pred))

Accuracy: 0.67
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.48      0.65       125
           1       0.53      0.99      0.69        75

    accuracy                           0.67       200
   macro avg       0.76      0.73      0.67       200
weighted avg       0.81      0.67      0.66       200

